In [1]:
import os 
import glob

1. Read in Data and make it manageable

In [2]:
#Look for all txt files in specific directory, read them in and append each song in list called corpus
file_list = glob.glob(os.path.join(os.getcwd(), './data/Manowar', "*.txt"))

manowar = []

for file_path in file_list:
    with open(file_path) as f_input:
        manowar.append(f_input.read())

# file_list = glob.glob(os.path.join(os.getcwd(), './data/Snoop Dogg', "*.txt"))

# snoop = []

# for file_path in file_list:
#     with open(file_path) as f_input:
#         snoop.append(f_input.read())

        

file_list = glob.glob(os.path.join(os.getcwd(), './data/Flogging Molly', "*.txt"))

molly = []

for file_path in file_list:
    with open(file_path) as f_input:
        molly.append(f_input.read())

file_list = glob.glob(os.path.join(os.getcwd(), './data/Queen', "*.txt"))

queen = []

for file_path in file_list:
    with open(file_path) as f_input:
        queen.append(f_input.read())

        
file_list = glob.glob(os.path.join(os.getcwd(), './data/Iron Maiden', "*.txt"))

maiden = []

for file_path in file_list:
    with open(file_path) as f_input:
        maiden.append(f_input.read())
        
        
file_list = glob.glob(os.path.join(os.getcwd(), './data/Blind Guardian', "*.txt"))

guardian = []

for file_path in file_list:
    with open(file_path) as f_input:
        guardian.append(f_input.read())
        guardian2=''

In [3]:
# artist=['Queen','Iron Maiden','Blind Guardian']

In [4]:
data={}
# for i,c in enumerate(artist):
#     file_list = glob.glob(os.path.join(os.getcwd(), f'./data/{artist}', "*.txt"))
#     for file_path in file_list:
#         with open(file_path) as f_input:
#             print(f_input)
            
            


In [5]:
data={'Blind Guardian':guardian,'Iron Maiden' : maiden,'Queen' : queen,'Flogging Molly':molly,'Manowar':manowar}

In [6]:
# data['Snoop Dogg']

In [7]:
# next(iter(data.keys()))

In [8]:
# next(iter(data.values()))

2 . Clean the Data!

Lower Case, Special Chars, numerical, tokenize, lemma,stemming, stopwords

In [9]:
import pandas as pd

In [10]:
def combine_text(list_of_text):
    combined_text=''.join(list_of_text)
    return combined_text

In [11]:
combined_data = {key: [combine_text(value)] for (key, value) in data.items()}

In [12]:
df=pd.DataFrame.from_dict(combined_data).transpose()

In [13]:
df.columns=['Lyrics']        

In [14]:
import spacy
import re
import string
model = spacy.load('en_core_web_md') 


In [15]:
def cleaning_function(text):
    text = text.lower()    # make all lower case
    text = re.sub('\[.*?\]','',text)  #Gets rid from data in brackets, like different singer --> if square bracket, get rid
    text = re.sub('[%s]' % re.escape(string.punctuation),'',text) #gets rid of the whole punctuation , . : etc.
    text = re.sub('\w*\d\w*', '',text) #If you have a number and there are any letters/numbers surrounding, kick out
    clean_text = []
    token_text = model(text)
    for word in token_text:
        if not word.is_stop and not word.is_punct and word.is_alpha:
            clean_text.append(word.lemma_)
    return ' '.join(clean_text)

In [28]:
df_clean=pd.DataFrame(df.Lyrics.apply(cleaning_function))

In [29]:
df_clean.index.rename('artist',inplace=True)

In [30]:
# df_clean.Lyrics.loc['Queen']  # looks good 

In [32]:
# df_clean

In [33]:
import pickle

In [34]:
df_clean.to_pickle("corpus.pkl")     # Export to pickle for use later

Count Vectorizer

In [35]:
from sklearn.feature_extraction.text import CountVectorizer

In [36]:
cv=CountVectorizer(stop_words='english')   #instantiate it with englisch stop words, for dropping the columns
data_cv=cv.fit_transform(df_clean.Lyrics)
data_dtm=pd.DataFrame(data_cv.toarray(),columns=cv.get_feature_names())
data_dtm.index=df_clean.index
data_dtm

,aaa,aaaaaaaarghh,aaah,aaargh,aah,abandon,abate,abel,abide,ablaze,...,zeigt,zero,zeus,ziel,zone,zoo,zoomin,zu,zum,zurück
artist,,,,,,,,,,,,,,,,,,,,,
Blind Guardian,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
Iron Maiden,0,0,0,0,0,1,1,0,1,3,...,0,2,0,0,2,0,0,0,0,0
Queen,1,1,1,1,21,1,0,1,0,0,...,0,1,0,0,0,2,1,0,0,0
Flogging Molly,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Manowar,0,0,0,0,0,0,0,0,0,0,...,1,0,3,1,0,0,0,4,1,1


In [37]:
data_dtm.to_pickle("dtm.pkl")

In [38]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(df, test_size=0.2)